### Model

In [1]:
import os
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, Lambda, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras import regularizers
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from imgaug import augmenters as iaa
import cv2
import ntpath

In [2]:
!git clone https://github.com/tzudir/Self_driving

Cloning into 'Self_driving'...
remote: Enumerating objects: 33831, done.
remote: Counting objects: 100% (9714/9714), done.
remote: Compressing objects: 100% (9676/9676), done.
remote: Total 33831 (delta 46), reused 9691 (delta 32), pack-reused 24117
Receiving objects: 100% (33831/33831), 418.24 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Updating files: 100% (33728/33728), done.


In [3]:
datadir = '/content/Self_driving/data'
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(datadir, 'driving_log.csv'), names = columns)
pd.set_option('display.max_colwidth', -1)

<ipython-input-3-04b2e98e549a>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [4]:
data.head(5)

,center,left,right,steering,throttle,reverse,speed
0,C:\Users\realj\Documents\Car\IMG\center_2024_01_25_22_48_25_466.jpg,C:\Users\realj\Documents\Car\IMG\left_2024_01_25_22_48_25_466.jpg,C:\Users\realj\Documents\Car\IMG\right_2024_01_25_22_48_25_466.jpg,0.0,0.0,0.0,0.000091
1,C:\Users\realj\Documents\Car\IMG\center_2024_01_25_22_48_25_543.jpg,C:\Users\realj\Documents\Car\IMG\left_2024_01_25_22_48_25_543.jpg,C:\Users\realj\Documents\Car\IMG\right_2024_01_25_22_48_25_543.jpg,0.0,0.0,0.0,0.000116
2,C:\Users\realj\Documents\Car\IMG\center_2024_01_25_22_48_25_622.jpg,C:\Users\realj\Documents\Car\IMG\left_2024_01_25_22_48_25_622.jpg,C:\Users\realj\Documents\Car\IMG\right_2024_01_25_22_48_25_622.jpg,0.0,0.0,0.0,0.000132
3,C:\Users\realj\Documents\Car\IMG\center_2024_01_25_22_48_25_700.jpg,C:\Users\realj\Documents\Car\IMG\left_2024_01_25_22_48_25_700.jpg,C:\Users\realj\Documents\Car\IMG\right_2024_01_25_22_48_25_700.jpg,0.0,0.0,0.0,0.000161
4,C:\Users\realj\Documents\Car\IMG\center_2024_01_25_22_48_25_769.jpg,C:\Users\realj\Documents\Car\IMG\left_2024_01_25_22_48_25_769.jpg,C:\Users\realj\Documents\Car\IMG\right_2024_01_25_22_48_25_769.jpg,0.0,0.0,0.0,0.000227


In [5]:
def path_leaf(path):
  head, tail = ntpath.split(path)
  return tail
data['center'] = data['center'].apply(path_leaf)
data['left'] = data['left'].apply(path_leaf)
data['right'] = data['right'].apply(path_leaf)

In [6]:
data.head()

,center,left,right,steering,throttle,reverse,speed
0,center_2024_01_25_22_48_25_466.jpg,left_2024_01_25_22_48_25_466.jpg,right_2024_01_25_22_48_25_466.jpg,0.0,0.0,0.0,0.000091
1,center_2024_01_25_22_48_25_543.jpg,left_2024_01_25_22_48_25_543.jpg,right_2024_01_25_22_48_25_543.jpg,0.0,0.0,0.0,0.000116
2,center_2024_01_25_22_48_25_622.jpg,left_2024_01_25_22_48_25_622.jpg,right_2024_01_25_22_48_25_622.jpg,0.0,0.0,0.0,0.000132
3,center_2024_01_25_22_48_25_700.jpg,left_2024_01_25_22_48_25_700.jpg,right_2024_01_25_22_48_25_700.jpg,0.0,0.0,0.0,0.000161
4,center_2024_01_25_22_48_25_769.jpg,left_2024_01_25_22_48_25_769.jpg,right_2024_01_25_22_48_25_769.jpg,0.0,0.0,0.0,0.000227


In [7]:
num_bins = 25
samples_per_bin = 400
hist, bins = np.histogram(data['steering'], num_bins)
center = (bins[:-1]+ bins[1:]) * 0.5
# plt.bar(center,hist, width=0.05)
# plt.plot((np.min(data['steering']), np.max(data['steering'])),(samples_per_bin, samples_per_bin))

In [8]:
remove_list = []
for j in range(num_bins):
  list_ = []
  for i in range(len(data['steering'])):
    if data['steering'][i] >= bins[j] and data['steering'][i] <= bins[j+1]:
      list_.append(i)
  list_ = shuffle(list_)
  list_ = list_[samples_per_bin:]
  remove_list.extend(list_)

print('removed: ', len(remove_list))
data.drop(data.index[remove_list], inplace=True)
print('remaining: ', len(data))

removed:  1538
remaining:  1667


In [9]:
hist, bins = np.histogram(data['steering'], (num_bins))
# plt.bar(center,hist, width=0.05)
# plt.plot((np.min(data['steering']), np.max(data['steering'])),(samples_per_bin, samples_per_bin))

In [10]:
data.shape

(1667, 7)

In [11]:
def load_img_steering(datadir, df):
  image_path = []
  steering = []
  for i in range(len(data)):
    indexed_data = data.iloc[i]
    center, left, right = indexed_data[0], indexed_data[1], indexed_data[2]
    image_path.append(os.path.join(datadir, center.strip()))
    steering.append(float(indexed_data[3]))
    image_path.append(os.path.join(datadir,left.strip()))
    steering.append(float(indexed_data[3])+0.15)
    image_path.append(os.path.join(datadir,right.strip()))
    steering.append(float(indexed_data[3])-0.15)
  image_paths = np.asarray(image_path)
  steerings = np.asarray(steering)
  return image_paths, steerings

In [12]:
image_paths, steerings = load_img_steering(datadir + '/IMG', data)

In [13]:
image_paths

array(['/content/Self_driving/data/IMG/center_2024_01_25_22_48_25_543.jpg',
       '/content/Self_driving/data/IMG/left_2024_01_25_22_48_25_543.jpg',
       '/content/Self_driving/data/IMG/right_2024_01_25_22_48_25_543.jpg',
       ...,
       '/content/Self_driving/data/IMG/center_2024_02_02_09_53_08_948.jpg',
       '/content/Self_driving/data/IMG/left_2024_02_02_09_53_08_948.jpg',
       '/content/Self_driving/data/IMG/right_2024_02_02_09_53_08_948.jpg'],
      dtype='<U65')

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(image_paths, steerings, test_size=0.2, random_state = 6)
print('Training Samples: {}\nValid Samples: {}'.format(len(X_train), len(X_valid)))

Training Samples: 4000
Valid Samples: 1001


In [30]:
# fig, axes = plt.subplots(1, 2, figsize=(12, 4))
# axes[0].hist(y_train, bins=num_bins, width = 0.5, color='blue')
# axes[0].set_title("Training set")
# axes[1].hist(y_valid, bins=num_bins, width = 0.5, color='red')
# axes[1].set_title("Validation set")

In [16]:
def zoom(image):
  zoom = iaa.Affine(scale=(1, 1.3))
  image = zoom.augment_image(image)
  return image

def pan(image):
  pan = iaa.Affine(translate_percent= {"x" : (-0.1, 0.1), "y": (-0.1, 0.1)})
  image = pan.augment_image(image)
  return image

def img_random_brightness(image):
    brightness = iaa.Multiply((0.2, 1.2))
    image = brightness.augment_image(image)
    return image

def img_random_flip(image, steering_angle):
    image = cv2.flip(image,1)
    steering_angle = -steering_angle
    return image, steering_angle



In [17]:
def random_augment(image, steering_angle):
    image = mpimg.imread(image)
    if np.random.rand() < 0.5:
      image = pan(image)
    if np.random.rand() < 0.5:
      image = zoom(image)
    if np.random.rand() < 0.5:
      image = img_random_brightness(image)
    if np.random.rand() < 0.5:
      image, steering_angle = img_random_flip(image, steering_angle)

    return image, steering_angle

In [18]:
# ncol = 2
# nrow = 6

# fig, axs = plt.subplots(nrow, ncol, figsize=(15, 50))
# fig.tight_layout()

# for i in range(6):
#   randnum = random.randint(0, len(image_paths) - 1)
#   random_image = image_paths[randnum]
#   random_steering = steerings[randnum]

#   original_image = mpimg.imread(random_image)
#   augmented_image, steering = random_augment(random_image, random_steering)

#   # axs[i][0].imshow(original_image)
#   # axs[i][0].set_title("Original Image")

#   # axs[i][1].imshow(augmented_image)
#   # axs[i][1].set_title("Augmented Image")

In [19]:
def img_preprocess(img):
  img = img[60:135,:,:]
  img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
  img = cv2.GaussianBlur(img, (3, 3), 0)
  img = cv2.resize(img, (200, 66))
  img = img/255
  return img

In [20]:
def batch_generator(image_paths, steering_ang, batch_size, istraining):

  while True:
    batch_img = []
    batch_steering = []

    for i in range(batch_size):
      random_index = random.randint(0, len(image_paths) - 1)

      if istraining:
        im, steering = random_augment(image_paths[random_index], steering_ang[random_index])

      else:
        im = mpimg.imread(image_paths[random_index])
        steering = steering_ang[random_index]

      im = img_preprocess(im)
      batch_img.append(im)
      batch_steering.append(steering)
    yield (np.asarray(batch_img), np.asarray(batch_steering))

In [21]:
# def nvidia_model():
#   model = Sequential()
#   model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu'))
#   model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu'))
#   model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu'))
#   model.add(Conv2D(64, (5, 5), activation='elu'))
#   # model.add(Conv2D(64, (3,3), activation='elu'))
# #   model.add(Dropout(0.5))


#   model.add(Flatten())

#   model.add(Dense(100, activation = 'elu'))
# #   model.add(Dropout(0.5))

#   model.add(Dense(50, activation = 'elu'))
# #   model.add(Dropout(0.5))

#   model.add(Dense(10, activation = 'elu'))
# #   model.add(Dropout(0.5))

#   model.add(Dense(1))

#   optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
#   model.compile(loss='mse', optimizer=optimizer)
#   return model

# model = nvidia_model()
# print(model.summary())

In [22]:
# def keras_model():
#     model = Sequential()
#     model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(66, 200, 3)))
#     model.add(Conv2D(32, (3, 3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling2D((2, 2), padding='valid'))
#     model.add(Conv2D(32, (3, 3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling2D((2, 2), padding='valid'))

#     model.add(Conv2D(64, (3, 3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling2D((2, 2), padding='valid'))
#     model.add(Conv2D(64, (3, 3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling2D((2, 2), padding='valid'))

#     model.add(Conv2D(128, (3, 3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling2D((2, 2), padding='valid'))
#     model.add(Conv2D(128, (3, 3), padding='same'))
#     model.add(Activation('relu'))
#     model.add(MaxPooling2D((2, 2), padding='valid'))

#     model.add(Flatten())
#     model.add(Dropout(0.5))
#     model.add(Dense(1024))
#     model.add(Dense(256))
#     model.add(Dense(64))
#     model.add(Dense(1))

#     optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
#     model.compile(loss='mse', optimizer=optimizer)
#     return model

# model = keras_model()
# print(model.summary())

In [23]:
# def keras_model():
#     model = Sequential()
#     model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(66, 200, 3)))
#     model.add(Conv2D(24, (5, 5), strides=(2, 2), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Flatten())
#     model.add(Dropout(0.5))
#     model.add(Dense(100, activation='relu'))
#     model.add(Dense(50, activation='relu'))
#     model.add(Dense(10, activation='relu'))
#     model.add(Dense(1))

#     optimizer = Adam(learning_rate=1e-4)
#     model.compile(loss='mse', optimizer=optimizer)

#     return model

# model = keras_model()
# print(model.summary())

In [24]:
# def keras_model():
#     model = Sequential()
#     model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(66, 200, 3)))
#     model.add(Conv2D(24, (5, 5), strides=(2, 2), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(BatchNormalization())
#     model.add(Flatten())
#     model.add(Dropout(0.5))
#     model.add(Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
#     model.add(Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
#     model.add(Dense(10, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
#     model.add(Dense(1))

#     optimizer = Adam(learning_rate=1e-4)
#     model.compile(loss='mse', optimizer=optimizer)

#     return model

# model = keras_model()
# print(model.summary())

# # Define early stopping criteria
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [25]:
def keras_model():
  model = Sequential()
  model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu'))
  model.add(BatchNormalization())
  model.add(Conv2D(64, (5, 5), activation='elu'))
  # Add a skip connection here (consider from after the second convolutional layer)

  model.add(Flatten())
  model.add(Dense(100, activation='elu'))
  model.add(Dropout(0.2))  # Add dropout
  model.add(Dense(50, activation='elu'))
  model.add(Dropout(0.5))  # Add dropout
  model.add(Dense(10, activation='elu'))
  model.add(Dense(1))

  optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)  # Experiment with optimizer
  model.compile(loss='mse', optimizer=optimizer)
  return model

model = keras_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 98, 24)        1824      
                                                                 
 batch_normalization (Batch  (None, 31, 98, 24)        96        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 47, 36)        21636     
                                                                 
 batch_normalization_1 (Bat  (None, 14, 47, 36)        144       
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 22, 48)         43248     
                                                                 
 batch_normalization_2 (Bat  (None, 5, 22, 48)         1

In [26]:
# history = model.fit_generator(batch_generator(X_train, y_train, 100, 1),
#                                   steps_per_epoch=300,
#                                   epochs=10,
#                                   validation_data=batch_generator(X_valid, y_valid, 100, 0),
#                                   validation_steps=200,
#                                   verbose=1,
#                                   shuffle = 1,
#                                   callbacks=[early_stopping])

In [27]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.legend(['training', 'validation'])
# plt.title('Loss')
# plt.xlabel('Epoch')

In [28]:
# model.save('model.h5')
# from google.colab import files
# files.download('model.h5')